# Tutorial 2: Investigate h- and p- refinements for a large deformation problem

In this tutorial, we study the convergence of hyper-elstic beam subjected to a large load $q=2$

Consider the following beam with the given material and geometry properties

![Nonlinear_beam](non-linear-beam.jpeg)

One can modift how many elements to use in the following code:

In [1]:
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_path, "src"))


from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
tutorials_dir = Path().resolve()

# for saving files later
tutorials_dir = Path().resolve()

# --- Beam geometry ---
L = 8.0   # length in x
H = 1.0    # height in y
nx = 8    # number of elements along length
ny = 1     # number of elements along height

ele_type = "D2_nn8_quad"  # 2D, 4-node quadrilateral (linear)
ndof = 2                  # 2 DOFs per node (x, y)

# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)


# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

# 1) Clamp the left edge: fix x- and y-displacements = 0
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

# 2) Uniform downward traction on the top edge (y=H)
# Let q be negative in the y-direction
q = -0.3  # load per unit length in x
# For a 2D plane strain problem, this is a traction (tx, ty) = (0, q)
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

# Combine boundary conditions
fixed_nodes = fixed_left  # only the left edge is clamped

# --- Material properties ---
E = 80000.0
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")

# Number of incremental load steps
nr_num_steps = 10

# --- Solve with your hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # shape (2, n_nodes)
    connect.T,     # shape (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-10,
    nr_maxit=30,
)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

# --- Compute the tip displacement from the FEA result ---
# We'll pick a node near x=L, y=H/2
tip_node = None
tol = 1e-3
for i, (x, y) in enumerate(coords):
    if abs(x - L) < tol and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break
if tip_node is None:
    raise ValueError("Could not find tip node near x=L, y=H/2.")

tip_disp_y = final_disp[ndof*tip_node + 1]  # the y-component of displacement


Material properties: mu=30769.231, kappa=66666.667
Step 0, load factor = 0.100
Iteration 1, Correction=1.000000e+00, Residual=7.284863e-04, tolerance=1.000000e-10
Iteration 2, Correction=1.440917e-04, Residual=2.450289e-04, tolerance=1.000000e-10
Iteration 3, Correction=2.587529e-09, Residual=1.507299e-11, tolerance=1.000000e-10
Iteration 4, Correction=8.634274e-13, Residual=5.193359e-13, tolerance=1.000000e-10
Step 1, load factor = 0.200
Iteration 1, Correction=5.000119e-01, Residual=7.284863e-04, tolerance=1.000000e-10
Iteration 2, Correction=7.203922e-05, Residual=2.450816e-04, tolerance=1.000000e-10
Iteration 3, Correction=1.291958e-09, Residual=1.508127e-11, tolerance=1.000000e-10
Iteration 4, Correction=7.170987e-14, Residual=4.118037e-13, tolerance=1.000000e-10
Step 2, load factor = 0.300
Iteration 1, Correction=3.333544e-01, Residual=7.284863e-04, tolerance=1.000000e-10
Iteration 2, Correction=4.802175e-05, Residual=2.451342e-04, tolerance=1.000000e-10
Iteration 3, Correction=8

The tip is calculated to be:

In [7]:
print(f"Tip node index: {tip_node}, coordinates={coords[tip_node]}")
print(f"Computed tip deflection (y): {tip_disp_y:.6f}")


Tip node index: 25, coordinates=[8.  0.5]
Computed tip deflection (y): -0.020755


We plot the reference solution that we get from a very fine mesh against the ifferent solutions that we get. The finer the mesh (i.e. the smaller the element), the closer we get to the reference solution.

![h-refinement](h-refinement.jpeg)

We plot the reference solution that we get from the best mesh against the different solutions that we get using diffrent polynomial orders with same number of elements. The finer the mesh (i.e. the higher the order), the closer we get to the reference solution.

![p-refinement](p-refinement.png)